In [10]:
import nest_asyncio

import autogen
from autogen.agents.experimental import WebSurferAgent

nest_asyncio.apply()

In [11]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4o-mini"]},
)
llm_config = {"config_list": config_list}

In [12]:
from typing import Annotated, Any, List, Optional
from autogen import ConversableAgent
from autogen.tools.dependency_injection import BaseContext, Depends
from pydantic import BaseModel

# Swarm Agents, similar to ConversableAgent, but with functions and hand offs (specified later)
initial_analyser = ConversableAgent(
    name="InitialAnalysisAgent",
    system_message="""You are an expert at breaking down complex questions into smaller, focused subquestions.
Your task is to take any question provided and divide it into clear, actionable subquestions that can be individually answered.
Ensure the subquestions are logical, non-redundant, and cover all key aspects of the original question.
Avoid providing answers or interpretations—focus solely on decomposition.

Do NOT forward the initial task without breaking it down into subquestions!!!
""",
    llm_config=llm_config,
)

class Subtask(BaseModel):
    question: str
    answer: Optional[str] = None

class Subtasks(BaseContext, BaseModel):
    in_progress_subtask: Optional[Subtask] = None
    subtasks: List[Subtask]

subtasks_context: Subtasks = Subtasks(subtasks=[])

@initial_analyser.register_for_llm(description="Decompose the question into subtasks with the maximum of three subtasks.")
def decompose_question(subtasks: list[str], subtasks_context: Annotated[Subtasks, Depends(subtasks_context)]) -> None:
    subtasks_context.subtasks.extend([Subtask(question=st) for st in subtasks])

In [13]:
websurfer = WebSurferAgent(
    llm_config=llm_config,
    name="WebSurferAgent",
    system_message="""You are a web surfer agent responsible for gathering information from the web to answer the subquestions provided.
Always perform searches one subquestion at a time. You are allowed to perform a new search only after you receive a new subquestion.
""",
    # web_tool="crawl4ai",
)

@websurfer.register_for_llm(description="Get next subtask")
def get_next_subtask(subtasks_context: Annotated[Subtasks, Depends(subtasks_context)]) -> str:
    if subtasks_context.in_progress_subtask:
        return f"Subtask already in progress, first answer previous subtask: {subtasks_context.in_progress_subtask.question}"
    
    unanswered_subtasks = [st for st in subtasks_context.subtasks if st.answer is None]
    
    if not unanswered_subtasks:
        return "No more subtasks, proceed to the next agent"
    
    subtasks_context.in_progress_subtask = unanswered_subtasks[0]
    
    return subtasks_context.in_progress_subtask.question

@websurfer.register_for_llm(description="Answer the subtask")
def answer_subtask(answer: str, subtasks_context: Annotated[Subtasks, Depends(subtasks_context)]) -> str:
    if not subtasks_context.in_progress_subtask:
        return "No subtask in progress, first get a subtask"
    
    subtasks_context.in_progress_subtask.answer = answer
    subtasks_context.in_progress_subtask = None

    return "Subtask answered successfully"

In [14]:
summary_agent = ConversableAgent(
    name="SummaryAgent",
    system_message="""You are a summarizer agent responsible for summarizing the answers to the subquestions provided.
Your task is to provide a concise summary of the answers to the subquestions.
Ensure the summary is coherent, logically structured, and covers all key aspects of the original question.
""",
    llm_config=llm_config,
)

In [15]:
from autogen.agentchat.user_proxy_agent import UserProxyAgent


user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="A proxy to excute code",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    code_execution_config=False,
    function_map={
        "answer_subtask": answer_subtask,
        "get_next_subtask": get_next_subtask,
        "decompose_question": decompose_question,
    },
)

user_proxy.register_for_execution()(websurfer.tools[0])

In [16]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    # always start with the user
    if len(messages) <= 1:
        return initial_analyser
    
    print(messages[-1])

    # if the last message is a tool call, return the tool_execution agent
    if "tool_calls" in messages[-1]:
        return user_proxy

    # now, we define what the next agent should be based on the last speaker
    # the best practice is to go through each agent and think about the possible transitions
    if last_speaker is user_proxy:
        penultimate_speaker = groupchat.agent_by_name(name=messages[-2].get("name", ""))
        
        if penultimate_speaker is initial_analyser:
            return websurfer
    
        if messages[-1].get("content", "").startswith("No more subtasks"):
            return summary_agent

        return groupchat.agent_by_name(name=messages[-2].get("name", ""))
        


    if last_speaker is initial_analyser:
        return websurfer
    
    if last_speaker is websurfer:
        return summary_agent

In [17]:
groupchat = autogen.GroupChat(
    agents=[initial_analyser, websurfer, summary_agent, user_proxy],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

chat_result = user_proxy.initiate_chat(
    manager,
    message="Which are the best cryptocurrencies to invest in? Do the web research first and make som conclusions",  # Initial message
)

user_proxy (to chat_manager):

Which are the best cryptocurrencies to invest in? Do the web research first and make som conclusions

--------------------------------------------------------------------------------

Next speaker: InitialAnalysisAgent


>>>>>>>> USING AUTO REPLY...
InitialAnalysisAgent (to chat_manager):

***** Suggested tool call (call_kXDqfcwRgAem3zJOcdwFfxML): decompose_question *****
Arguments: 
{"subtasks":["What criteria should be used to evaluate the best cryptocurrencies for investment?","What are the current market trends and performance indicators for top cryptocurrencies?","Which cryptocurrencies have shown consistent growth or stability in recent years?"]}
***********************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION decompose_question...
Call ID: call_kXDqfcwRgAem3zJOcdwFfxML
Input arguments: {'subtas

KeyboardInterrupt: 

INFO     [controller] 🔍  Searched for "criteria to evaluate best cryptocurrencies for investment" in Google
INFO     [agent] 📍 Step 2
INFO     [agent] 🤷 Eval: Unknown - No previous actions to evaluate.
INFO     [agent] 🧠 Memory: 
INFO     [agent] 🎯 Next goal: Click on 'How to evaluate a cryptocurrency' from Fidelity Investments.
INFO     [agent] 🛠️  Action 1/1: {"click_element":{"index":1}}
INFO     [controller] 🖱️  Clicked button with index 1: 
INFO     [agent] 📍 Step 3
INFO     [agent] 🤷 Eval: Unknown - No previous actions to evaluate.
INFO     [agent] 🧠 Memory: 
INFO     [agent] 🎯 Next goal: Click on '7 Key Ways to Evaluate a Cryptocurrency Before Buying It' from CoinDesk.
INFO     [agent] 🛠️  Action 1/1: {"click_element":{"index":6}}
INFO     [controller] 🖱️  Clicked button with index 6: ‪Français (France)‬
INFO     [agent] 📍 Step 4
INFO     [agent] 🤷 Eval: Unknown - No previous actions to evaluate.
INFO     [agent] 🧠 Memory: 
INFO     [agent] 🎯 Next goal: Go back to the search res